In [3]:
import pickle
import pandas as pd
import sys

from gensim.models.word2vec import Word2Vec

sys.path.append('..')

from tree import ASTNode, SingleNode 
from prepare_data import get_blocks

In [ ]:
def read_ast(language):
  root = 'data/'
  source = pd.read_pickle(root+language+'/ast.pkl')
  print(len(source['code']))
  return source

In [ ]:
language = 'c'
source_c_ast_DF = read_ast(language)

### 读取训练集

In [ ]:
root = 'data/'
language = 'c'
train_file_path = root + language + '/train/train.pkl'
pairs = pd.read_pickle(train_file_path)
# 找到训练集所包含的所有的source code id
train_ids = pairs['id1'].append(pairs['id2']).unique()

### 载入训练好的词典

In [ ]:
embedding_size = 128
word2vec = Word2Vec.load(root+language+'/train/embedding_astnode_w2v_' + str(embedding_size)).wv
vocab = word2vec.vocab
max_token = word2vec.syn0.shape[0]

In [ ]:
def tree_to_index(node):
    token = node.token
    result = [vocab[token].index if token in vocab else max_token]
    children = node.children
    for child in children:
        #print(f'tree_to_index: a node''s child: {child.token}')
        result.append(tree_to_index(child))
    return result

In [ ]:
def trans2seq(ast):
    blocks = []
    # 获得语句树的集合
    get_blocks(ast, blocks)
    tree = []

    # 对每一个语句树的节点表示进行word2vec转换
    for b in blocks:
        # btree每一个语句树的word2vec表示
        btree = tree_to_index(b)
        tree.append(btree)
    return tree

In [ ]:
source_2_seq_DF =  source_c_ast_DF.copy(False)
trees = source_2_seq_DF['code']
source_2_seq_DF['code'] = trees.apply(trans2seq)
if 'label' in source_2_seq_DF.columns:
        source_2_seq_DF.drop('label', axis=1, inplace=True)
blocks = source_2_seq_DF